In [ ]:
import numpy as np
import glob
import h5py
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc
import glob

%matplotlib inline
plt.rcParams["figure.figsize"] = (7,5)

In [ ]:
models = (
    ("Graph AE", h5py.File("./output/garnet_mse/output.h5", 'r')),
    ("Convolutional AE", h5py.File("./output/cnn/output.h5", 'r')),
)

signals_loc = glob.glob("./signals/*")

In [ ]:
def plot_training_history(model_output, title):
    loss = model_output['loss'][:]
    val_loss = model_output['val_loss'][:]
    
    plt.figure()
    plt.plot(loss, color="crimson")
    plt.plot(val_loss, color="tab:green")
    plt.title("Training History - %s"%title)

    plt.semilogy()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.tight_layout()
    plt.grid(True)

for model in models:
    plot_training_history(model[1], model[0])

In [ ]:
def plot_features(model_output, title, signals_loc):
    features = ("eta", "phi", "pT")

    for f in ["QCD"]+signals_loc:
        if f == "QCD":
            qcd = model_output[f][()]
        else:
            print(f)
            qcd = h5py.File(f, 'r')["jetConstituentsList"][()]
            
        predicted_qcd = model_output["predicted_"+f][()]

        num_of_jets = qcd.shape[1]

        for i, feature in enumerate(features):
            plt.hist(np.squeeze(qcd[:,:,i]), 50, label="Input %s"%feature, density=True, 
                histtype='step', fill=False, linewidth=1.5, color=["dodgerblue" for x in range(num_of_jets)])

            plt.hist(np.squeeze(predicted_qcd[:,:,i]), 50, label="Reconstruced %s"%feature, density=True,
                histtype='step', fill=False, linewidth=1.5, color=["firebrick" for x in range(num_of_jets)])
            
            plt.yscale('log', nonposy='clip')     
            plt.xlabel(feature, fontsize=15)
            plt.ylabel('Prob. Density (a.u.)', fontsize=15)
            plt.title("%s - %s"%(title, f))
            plt.grid(True)
            plt.legend(loc="upper right")

            plt.show()

for model in models:
    plot_features(model[1], model[0], signals_loc)

In [ ]:
def mse_loss(inputs, outputs):
    return np.mean((inputs-outputs)*(inputs-outputs), axis=-1)

loss = mse_loss

In [ ]:
def ae_loss(model_output, title, signals_loc):
    plt.figure()

    for f in ["QCD"]+signals_loc:
        
        if f == "QCD":
            fill, alpha = [True, 0.2]
            inval = model_output[f][()]
            name = f
        else:
            fill, alpha = [False, 1]
            inval = h5py.File(f, 'r')["jetConstituentsList"][()]
            name = Path(f).stem

        outval = model_output["predicted_"+name][()]

        lossQCD = loss(np.reshape(inval, [-1, 16*3*1]), np.reshape(outval, [-1, 16*3*1]))
        maxScore = np.max(lossQCD)

        # plot AE Loss
        plt.hist(lossQCD, bins=100, label=name, density=True, range=(0, maxScore), 
                 histtype='step', fill=fill, alpha=alpha, linewidth=1.5)

    plt.semilogy()
    plt.title(title)
    plt.xlabel("AE Loss")
    plt.ylabel("Probability (a.u.)")
    plt.grid(True)
    plt.legend(loc="upper right")
    plt.show()

for model in models:
    ae_loss(model[1], model[0], signals_loc)

In [ ]:
def metrics(model_output, signals_loc):
    lossQCD = loss(np.reshape(model_output["QCD"][()], [-1, 16*3*1]), np.reshape(model_output["predicted_QCD"][()], [-1, 16*3*1]))
    
    metrics = []
    
    for f in signals_loc:
        name = Path(f).stem
        
        anomaly = h5py.File(f, 'r')["jetConstituentsList"][()]
        loss_anomaly = loss(np.reshape(anomaly, [-1, 16*3*1]), np.reshape(model_output["predicted_"+name][()], [-1, 16*3*1]))

        true_val = np.concatenate((np.ones(loss_anomaly.shape[0]), np.zeros(lossQCD.shape[0])))
        pred_val = np.concatenate((loss_anomaly, lossQCD))

        fpr, tpr, threshold = roc_curve(true_val, pred_val)
        auc_data = auc(fpr, tpr)
        
        metrics.append([name, fpr, tpr, auc_data])
    
    return np.array(metrics, dtype=object)

In [ ]:
def plot_roc(model_output, title, signals_loc):
    for sig_metrics in metrics(model_output, signals_loc):

        plt.plot(sig_metrics[2], sig_metrics[1], label="%s (auc = %.1f%%)"%(sig_metrics[0], sig_metrics[3]*100.))

        plt.semilogy()
        plt.title(title)
        plt.xlabel("sig. efficiency")
        plt.ylabel("bkg. mistag rate")
        plt.grid(True)
        plt.legend(loc="lower right")
         
    plt.show()
    
for model in models:
    plot_roc(model[1], model[0], signals_loc)

In [ ]:
def ratios(base_output, signals_loc):
    model_output = h5py.File(base_output, "r")
    base = metrics(model_output, signals_loc)
    plt.hlines(1, 1, 17, linestyles='--', color='#ef5675', linewidth=1.5)
    plt.ylim(0.7, 1.3)
    
    for i in range(2, 17, 2):
        model_output_qp = h5py.File("output/cnn_qp"+str(i)+"/output.h5", "r")
        bits = metrics(model_output_qp, signals_loc)
        plt.scatter([i]*len(base), bits[:,3]/base[:,3])
        
    # plt.legend()
    plt.xlabel("Bit Width")
    plt.ylabel("Sig. AUC / Base AUC")
    plt.title("Bit Ratios")
    plt.show()
    
ratios("output/cnn/output.h5", signals_loc)